SYLOGISM TRAINING USING GRPO --- LOG SAVED IN wordle_log.txt

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from peft import LoraConfig, get_peft_model
ckpt = "google/gemma-3-4b-it"

model = AutoModelForImageTextToText.from_pretrained(
    ckpt, device_map="auto", torch_dtype=torch.bfloat16, attn_implementation="eager"
)

# Load LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

processor = AutoProcessor.from_pretrained(ckpt)

/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


trainable params: 38,497,792 || all params: 4,338,577,264 || trainable%: 0.8873
None


Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 31536.12it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
import re
from datasets import load_dataset, Dataset

SYSTEM_PROMPT = """
You are playing Wordle, a word-guessing game.

Think through the problem and feedback step by step. Make sure to first add your step by step thought process within <think> </think> tags. Then, return your guessed word in the following format: <guess> guessed-word </guess>.

Important: Your guess must always respect the previous feedback: all letters marked ✓ must be in the same position, letters marked - must be included but not in that position, and letters marked x must NOT appear in your guess.
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<guess>")[-1]
    answer = answer.split("</guess>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()


def get_wordle_dataset(split="train") -> Dataset:
    data = load_dataset("predibase/wordle-grpo", split=split)
    
    # Prepare the prompt-answer pairs
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['prompt']}  # Use the full prompt from the dataset
        ],
        'answer': x['secret']  # secret word as the final answer to guess
    })
    return data

dataset = get_wordle_dataset()

Map: 100%|██████████| 76/76 [00:00<00:00, 10464.76 examples/s]


In [4]:
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_guesses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Prompt:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted Guess:\n{extracted_guesses[0]}")
    return [2.0 if guess.lower() == ans.lower() else 0.0 for guess, ans in zip(extracted_guesses, answer)]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Checks strict format: <think> ... </think>\n<guess> ... </guess>"""
    pattern = r"^<think>\n.*?\n</think>\n<guess>\n.*?\n</guess>$"
    responses = [completion[0]['content'] for completion in completions]
    matches = [re.match(pattern, r, re.DOTALL) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Checks that both <think> and <guess> tags appear somewhere"""
    pattern = r"<think>.*?</think>.*?<guess>.*?</guess>"
    responses = [completion[0]['content'] for completion in completions]
    matches = [re.search(pattern, r, re.DOTALL) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<think>\n") == 1:
        count += 0.125
    if text.count("\n</think>\n") == 1:
        count += 0.125
    if text.count("\n<guess>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</guess>\n")[-1]) * 0.001
    if text.count("\n</guess>") == 1:
        count += 0.125
        count -= (len(text.split("\n</guess>")[-1]) - 1) * 0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]['content'] for completion in completions]
    return [count_xml(c) for c in contents]




In [6]:
from trl import GRPOConfig, GRPOTrainer

max_prompt_length = 256
max_seq_length = 1024

training_args = GRPOConfig(
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 1,
    num_generations = 2,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 1,
    max_steps = 600,        #250
    save_steps = 100,       #250
    max_grad_norm = 0.1,
    report_to = "none",
)

In [7]:
trainer = GRPOTrainer(
    model=model,
    processing_class=processor,
    reward_funcs=[
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        correctness_reward_func,
    ],
    args=training_args,
    train_dataset=dataset,
)

trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.
`generation_config` default values have been modified to match model-specific defaults: {'top_p': 0.95}. If this is not desired, please set these values explicitly.


-------------------- Prompt:
<|im_start|>system

You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to first add your step by step thought process within <think> </think> tags. Then, return your guessed word in the following format: <guess> guessed-word </guess>.
<|im_en

Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


-------------------- Prompt:
<|im_start|>system

You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to first add your step by step thought process within <think> </think> tags. Then, return your guessed word in the following format: <guess> guessed-word </guess>.
<|im_en

TrainOutput(global_step=600, training_loss=-0.00048784380157788595, metrics={'train_runtime': 31891.8317, 'train_samples_per_second': 0.038, 'train_steps_per_second': 0.019, 'total_flos': 0.0, 'train_loss': -0.00048784380157788595})

 PREMISE - CONCLUSION TRAINING USING GRPO --- LOG SAVED IN log_conclusion_1.txt


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [12]:
import os
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOTrainer, GRPOConfig
from datasets import Dataset

# ✅ NEW: PEFT (LoRA)
from peft import get_peft_model, LoraConfig, TaskType

# ========== STEP 1: Define your logic dataset ==========
# logic_data = [
#     {"prompt": "All humans are mortal. Socrates is a human. Therefore,", "answer": "Socrates is mortal."},
#     {"prompt": "All birds can fly. A penguin is a bird. Therefore,", "answer": "A penguin can fly."},  # Wrong logic
#     {"prompt": "All cats are animals. A tiger is a cat. Therefore,", "answer": "A tiger is an animal."},
#     {"prompt": "If it rains, the ground gets wet. It is raining. Therefore,", "answer": "The ground gets wet."},
#     {"prompt": "All fish live in water. A shark is a fish. Therefore,", "answer": "A shark lives in water."},
# ]

logic_data = [
    {
        "prompt": "Given the premises: 'All humans are mortal' and 'Socrates is a human', write the logical conclusion in one sentence.",
        "answer": "Socrates is mortal."
    },
    {
        "prompt": "Given the premises: 'All birds can fly' and 'A penguin is a bird', write the logical conclusion in one sentence.",
        "answer": "A penguin can fly."  # still logically incorrect but part of dataset for training reward
    },
    {
        "prompt": "Given the premises: 'All cats are animals' and 'A tiger is a cat', write the logical conclusion in one sentence.",
        "answer": "A tiger is an animal."
    },
    {
        "prompt": "Given the premises: 'If it rains, the ground gets wet' and 'It is raining', write the logical conclusion in one sentence.",
        "answer": "The ground gets wet."
    },
    {
        "prompt": "Given the premises: 'All fish live in water' and 'A shark is a fish', write the logical conclusion in one sentence.",
        "answer": "A shark lives in water."
    },
]


expanded_data = logic_data * 25  # Now 125 examples
random.shuffle(expanded_data)

dataset = Dataset.from_list([{"prompt": d["prompt"], "answer": d["answer"]} for d in expanded_data])

# ========== STEP 2: Load model and tokenizer ==========
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_name)

# ========== STEP 3: Apply LoRA via PEFT ==========
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],  # or use ["c_attn"] for GPT-style
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()  # ✅ Shows how few parameters are being trained

# ========== STEP 4: Define reward function ==========
# def reward_func(prompts, completions, completion_ids, **kwargs):
#     rewards = []
#     for completion in completions:
#         output = completion.strip()
#         match_found = any(output.endswith(data["answer"]) for data in logic_data)
#         rewards.append(1.0 if match_found else 0.0)
#     return rewards

# def reward_func(prompts, completions, completion_ids, **kwargs):
#     rewards = []
    
#     for i, (prompt, completion) in enumerate(zip(prompts, completions)):
#         output = completion.strip().lower()
#         match_found = any(output.endswith(data["answer"]) for data in logic_data)
#         reward = 1.0 if match_found else 0.0
#         rewards.append(reward)

#         # 🔄 Print all completions for each prompt
#         print("=" * 50)
#         print(f"[Sample {i+1}]")
#         print(f"Prompt       : {prompt}")
#         print(f"Completion   : {output}")
#         print(f"Reward Score : {reward}")
#         print("=" * 50)

#     return rewards

import re

def clean_text(text):
    text = text.lower()
    # Remove common prefixes/tags
    text = re.sub(r'^(output|answer|completion|premises|ai|input|assistant|solution|response):\s*', '', text)
    # Remove punctuation except apostrophes inside words
    text = re.sub(r'[^\w\s\']', '', text)
    # Normalize whitespace
    text = ' '.join(text.split())
    return text

def reward_func(prompts, completions, completion_ids, **kwargs):
    rewards = []
    for i, completion in enumerate(completions):
        output_clean = clean_text(completion)
        matched = False
        for data in logic_data:
            answer_clean = clean_text(data["answer"])
            if answer_clean in output_clean:
                matched = True
                break
        reward = 1.0 if matched else 0.0

        print("="*50)
        print(f"[Sample {i+1}]")
        print(f"Prompt       : {prompts[i]}")
        print(f"Completion   : {completion.strip()}")
        print(f"Reward Score : {reward}")
        print("="*50)

        rewards.append(reward)
    return rewards




# ========== STEP 5: Configure GRPO training ==========
grpo_config = GRPOConfig(
    output_dir="grpo-logic-output",
    per_device_train_batch_size=1,
    generation_batch_size=2,
    num_train_epochs=2,
    learning_rate=1e-4,
    num_generations=2,  # group size
    logging_steps=5,
    remove_unused_columns=False,
    report_to=[],
    max_completion_length=20,
)

# ========== STEP 6: Initialize Trainer ==========
trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=dataset,
    reward_funcs=[reward_func],
)

# ========== STEP 7: Start training ==========
trainer.train()

# Save the LoRA adapter (not full model unless merged)
trainer.save_model("grpo-lora-model")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.28it/s]


trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[Sample 1]
Prompt       : Given the premises: 'All birds can fly' and 'A penguin is a bird', write the logical conclusion in one sentence.
Completion   : Output: The conclusion is 'Therefore, a penguin can fly.'
Reward Score : 1.0
[Sample 2]
Prompt       : Given the premises: 'All birds can fly' and 'A penguin is a bird', write the logical conclusion in one sentence.
Completion   : (2 points)

(c) Write three statements that would counterexample the claim
Reward Score : 0.0


Step,Training Loss
5,-0.053600
10,0.092000
15,0.099500
20,-0.064400
25,0.067500
30,-0.073600
35,0.087400
40,-0.070200
45,0.000000
50,0.000000


[Sample 1]
Prompt       : Given the premises: 'All birds can fly' and 'A penguin is a bird', write the logical conclusion in one sentence.
Completion   : Solution: The conclusion is 'Therefore, a penguin can fly.'

Example Follow
Reward Score : 1.0
[Sample 2]
Prompt       : Given the premises: 'All birds can fly' and 'A penguin is a bird', write the logical conclusion in one sentence.
Completion   : Input: 
Output: The logical conclusion is: 'A penguin can fly.'
Reward Score : 1.0
[Sample 1]
Prompt       : Given the premises: 'All fish live in water' and 'A shark is a fish', write the logical conclusion in one sentence.
Completion   : Input: 
Output: The logical conclusion is: 'A shark lives in water'.
Reward Score : 1.0
[Sample 2]
Prompt       : Given the premises: 'All fish live in water' and 'A shark is a fish', write the logical conclusion in one sentence.
Completion   : Premises: ['All dogs have tails', 'Fido is a dog'].
Output:
Reward Score : 0.0
[Sample 1]
Prompt       : Given t

In [ ]:
import os
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOTrainer, GRPOConfig
from datasets import Dataset

# ✅ NEW: PEFT (LoRA)
from peft import get_peft_model, LoraConfig, TaskType

# ========== STEP 1: Define your logic dataset ==========
# logic_data = [
#     {"prompt": "All humans are mortal. Socrates is a human. Therefore,", "answer": "Socrates is mortal."},
#     {"prompt": "All birds can fly. A penguin is a bird. Therefore,", "answer": "A penguin can fly."},  # Wrong logic
#     {"prompt": "All cats are animals. A tiger is a cat. Therefore,", "answer": "A tiger is an animal."},
#     {"prompt": "If it rains, the ground gets wet. It is raining. Therefore,", "answer": "The ground gets wet."},
#     {"prompt": "All fish live in water. A shark is a fish. Therefore,", "answer": "A shark lives in water."},
# ]

math_data = [
    {"prompt": "Solve the following and give just a number: What is 7 plus 5?", "answer": "12"},
    {"prompt": "Solve the following and give just a number: If you have 3 apples and you get 4 more, how many apples do you have?", "answer": "7"},
    {"prompt": "Solve the following and give just a number: Calculate the product of 6 and 9.", "answer": "54"},
    {"prompt": "Solve the following and give just a number: What is 15 divided by 3?", "answer": "5"},
    {"prompt": "Solve the following and give just a number: Subtract 8 from 20.", "answer": "12"},
    {"prompt": "Solve the following and give just a number: Add 14 and 28.", "answer": "42"},
    {"prompt": "Solve the following and give just a number: What is 9 times 7?", "answer": "63"},
    {"prompt": "Solve the following and give just a number: Divide 81 by 9.", "answer": "9"},
    {"prompt": "Solve the following and give just a number: If you have 20 candies and eat 5, how many are left?", "answer": "15"},
    {"prompt": "Solve the following and give just a number: Multiply 12 by 4.", "answer": "48"},
    {"prompt": "Solve the following and give just a number: What is the sum of 100 and 250?", "answer": "350"},
    {"prompt": "Solve the following and give just a number: Subtract 45 from 100.", "answer": "55"},
    {"prompt": "Solve the following and give just a number: If a car travels 60 miles in 1 hour, how far will it travel in 3 hours?", "answer": "180"},
    {"prompt": "Solve the following and give just a number: What is 8 multiplied by 8?", "answer": "64"},
    {"prompt": "Solve the following and give just a number: Divide 144 by 12.", "answer": "12"},
    {"prompt": "Solve the following and give just a number: Add 33 and 67.", "answer": "100"},
    {"prompt": "Solve the following and give just a number: If you buy 5 packs with 10 pencils each, how many pencils do you have?", "answer": "50"},
    {"prompt": "Solve the following and give just a number: What is 45 minus 19?", "answer": "26"},
    {"prompt": "Solve the following and give just a number: Multiply 7 by 11.", "answer": "77"},
    {"prompt": "Solve the following and give just a number: Divide 56 by 7.", "answer": "8"},
    {"prompt": "Solve the following and give just a number: Add 5 to 100.", "answer": "105"},
    {"prompt": "Solve the following and give just a number: Subtract 12 from 25.", "answer": "13"},
    {"prompt": "Solve the following and give just a number: What is 13 times 6?", "answer": "78"},
    {"prompt": "Solve the following and give just a number: Divide 90 by 10.", "answer": "9"},
    {"prompt": "Solve the following and give just a number: If a bus holds 40 people and 28 get on, how many seats remain?", "answer": "12"},
    {"prompt": "Solve the following and give just a number: What is 3 plus 14?", "answer": "17"},
    {"prompt": "Solve the following and give just a number: Multiply 9 by 5.", "answer": "45"},
    {"prompt": "Solve the following and give just a number: Divide 100 by 25.", "answer": "4"},
    {"prompt": "Solve the following and give just a number: Subtract 7 from 50.", "answer": "43"},
    {"prompt": "Solve the following and give just a number: Add 200 and 350.", "answer": "550"},
    {"prompt": "Solve the following and give just a number: What is the product of 14 and 3?", "answer": "42"},
    {"prompt": "Solve the following and give just a number: Divide 36 by 6.", "answer": "6"},
    {"prompt": "Solve the following and give just a number: If you have 8 packs of gum with 9 pieces each, how many pieces do you have?", "answer": "72"},
    {"prompt": "Solve the following and give just a number: Subtract 30 from 80.", "answer": "50"},
    {"prompt": "Solve the following and give just a number: Add 11 and 44.", "answer": "55"},
    {"prompt": "Solve the following and give just a number: Multiply 15 by 2.", "answer": "30"},
    {"prompt": "Solve the following and give just a number: Divide 60 by 5.", "answer": "12"},
    {"prompt": "Solve the following and give just a number: What is 7 minus 2?", "answer": "5"},
    {"prompt": "Solve the following and give just a number: Add 17 to 83.", "answer": "100"},
    {"prompt": "Solve the following and give just a number: Multiply 4 by 13.", "answer": "52"},
    {"prompt": "Solve the following and give just a number: Divide 72 by 9.", "answer": "8"},
    {"prompt": "Solve the following and give just a number: If a box contains 6 rows of 7 chocolates, how many chocolates are there?", "answer": "42"},
    {"prompt": "Solve the following and give just a number: Subtract 18 from 40.", "answer": "22"},
    {"prompt": "Solve the following and give just a number: Add 25 and 35.", "answer": "60"},
    {"prompt": "Solve the following and give just a number: Multiply 10 by 10.", "answer": "100"},
    {"prompt": "Solve the following and give just a number: Divide 48 by 8.", "answer": "6"},
    {"prompt": "Solve the following and give just a number: What is 9 plus 11?", "answer": "20"},
    {"prompt": "Solve the following and give just a number: Subtract 16 from 30.", "answer": "14"},
]


# expanded_data = logic_data * 25  # Now 125 examples
# random.shuffle(expanded_data)

# dataset = Dataset.from_list([{"prompt": d["prompt"], "answer": d["answer"]} for d in expanded_data])

# Option 2: Use original dataset directly for shorter training
dataset = Dataset.from_list([{"prompt": d["prompt"], "answer": d["answer"]} for d in math_data])


# ========== STEP 2: Load model and tokenizer ==========
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_name)

# ========== STEP 3: Apply LoRA via PEFT ==========
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],  # or use ["c_attn"] for GPT-style
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()  # ✅ Shows how few parameters are being trained


import re

def clean_text(text):
    text = text.lower()
    # Remove common prefixes/tags
    text = re.sub(r'^(output|answer|completion|premises|ai|input|assistant|solution|response):\s*', '', text)
    # Remove punctuation except apostrophes inside words
    text = re.sub(r'[^\w\s\']', '', text)
    # Normalize whitespace
    text = ' '.join(text.split())
    return text

def reward_func(prompts, completions, completion_ids, **kwargs):
    rewards = []
    for i, completion in enumerate(completions):
        output_clean = clean_text(completion)
        matched = False
        for data in math_data:
            answer_clean = clean_text(data["answer"])
            if answer_clean in output_clean:
                matched = True
                break
        reward = 1.0 if matched else 0.0

        print("="*50)
        print(f"[Sample {i+1}]")
        print(f"Prompt       : {prompts[i]}")
        print(f"Completion   : {completion.strip()}")
        print(f"Reward Score : {reward}")
        print("="*50)

        rewards.append(reward)
    return rewards

# ========== STEP 5: Configure GRPO training ==========
grpo_config = GRPOConfig(
    output_dir="grpo-logic-output",
    per_device_train_batch_size=1,
    generation_batch_size=2,
    num_train_epochs=5,
    learning_rate=1e-4,
    num_generations=2,  # group size
    logging_steps=5,
    remove_unused_columns=False,
    report_to=[],
    max_completion_length=20,
)

# ========== STEP 6: Initialize Trainer ==========
trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=dataset,
    reward_funcs=[reward_func],
)

# ========== STEP 7: Start training ==========
trainer.train()

# Save the LoRA adapter (not full model unless merged)
trainer.save_model("grpo-lora-model")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.32it/s]


trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[Sample 1]
Prompt       : Solve the following and give just a number: What is 9 times 7?
Completion   : ## INPUT
9 times 7
##OUTPUT
63
Reward Score : 1.0
[Sample 2]
Prompt       : Solve the following and give just a number: What is 9 times 7?
Completion   : 2.How much is 20% of 84? 
3. What is the result of
Reward Score : 1.0


Step,Training Loss
5,0.033100
10,0.000000
15,0.091100
20,0.006800
25,0.004200
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


[Sample 1]
Prompt       : Solve the following and give just a number: If you buy 5 packs with 10 pencils each, how many pencils do you have?
Completion   : Solution:
To find the total number of pencils, we need to multiply the number
Reward Score : 0.0
[Sample 2]
Prompt       : Solve the following and give just a number: If you buy 5 packs with 10 pencils each, how many pencils do you have?
Completion   : <|question_end|>Answer: 50
Reward Score : 1.0
[Sample 1]
Prompt       : Solve the following and give just a number: Calculate the product of 6 and 9.
Completion   : Li is driving to France from the UK.
She will drive a total distance of 275 miles
Reward Score : 1.0
[Sample 2]
Prompt       : Solve the following and give just a number: Calculate the product of 6 and 9.
Completion   : Output: 54
Reward Score : 1.0
[Sample 1]
Prompt       : Solve the following and give just a number: Divide 81 by 9.
Completion   : Answer:  9
Reward Score : 1.0
[Sample 2]
Prompt       : Solve the following

In [3]:
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOTrainer, GRPOConfig
from datasets import load_dataset, Dataset
from peft import get_peft_model, LoraConfig, TaskType

# ========== STEP 1: Load Wordle Dataset ==========

SYSTEM_PROMPT = "You are a helpful assistant solving Wordle puzzles."

def get_wordle_dataset(split="train"):
    data = load_dataset("predibase/wordle-grpo", split=split)

    # Add prompt with roles
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['prompt']}
        ],
        'answer': x['secret']
    })
    return data

def flatten_wordle_dataset(wordle_dataset):
    def combine_roles(example):
        system_content = example['prompt'][0]['content']
        user_content = example['prompt'][1]['content']
        combined_prompt = system_content + "\n" + user_content
        return {"prompt": combined_prompt, "answer": example["answer"]}
    
    return wordle_dataset.map(combine_roles, remove_columns=['prompt'])

# Prepare final dataset
dataset = flatten_wordle_dataset(get_wordle_dataset())
dataset = dataset.shuffle(seed=42)  # Optional shuffle

# ========== STEP 2: Load model and tokenizer ==========

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_name)

# ========== STEP 3: Apply LoRA via PEFT ==========

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

# ========== STEP 4: Reward Function ==========

def clean_text(text):
    text = text.lower()
    text = re.sub(r'^(output|answer|completion|premises|ai|input|assistant|solution|response):\s*', '', text)
    text = re.sub(r'[^\w\s\']', '', text)
    text = ' '.join(text.split())
    return text

# Updated reward function uses answer from dataset directly
def reward_func(prompts, completions, completion_ids, **kwargs):
    answers = kwargs.get("answers", [])  # List of ground-truth answers
    rewards = []

    for i, completion in enumerate(completions):
        output_clean = clean_text(completion)

        # Fetch correct answer using group index
        group_index = completion_ids[i][0] if isinstance(completion_ids[i], list) else completion_ids[i]
        expected_answer = clean_text(answers[group_index]) if group_index < len(answers) else ""

        reward = 1.0 if expected_answer in output_clean else 0.0

        print("="*50)
        print(f"[Sample {i+1}]")
        print(f"Prompt       : {prompts[i]}")
        print(f"Completion   : {completion.strip()}")
        print(f"Expected Ans : {expected_answer}")
        print(f"Reward Score : {reward}")
        print("="*50)

        rewards.append(reward)

    return rewards


# ========== STEP 5: GRPO Configuration ==========

grpo_config = GRPOConfig(
    output_dir="grpo-wordle-output",
    per_device_train_batch_size=1,
    generation_batch_size=2,
    num_train_epochs=5,
    learning_rate=1e-4,
    num_generations=2,  # group size
    logging_steps=5,
    remove_unused_columns=False,
    report_to=[],
    max_completion_length=20,
)

# ========== STEP 6: Initialize Trainer ==========

trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=dataset,
    reward_funcs=[reward_func],
)

# ========== STEP 7: Start Training ==========

trainer.train()

# Save the LoRA adapter (not full model unless merged)
trainer.save_model("grpo-lora-model")


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.95it/s]


trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[Sample 1]
Prompt       : You are a helpful assistant solving Wordle puzzles.
<|im_start|>system

You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to first add your step by step thought process within <think> </think> tags. Then, return your guessed word in the followi

Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


[Sample 1]
Prompt       : You are a helpful assistant solving Wordle puzzles.
<|im_start|>system

You are playing Wordle, a word-guessing game.

### Game Rules:
- You have **6 tries** to guess a secret **5-letter** word.
- Each guess must be a valid **5-letter English word**.
- After each guess, you will receive feedback indicating how close your guess was.

### Feedback Format:
Each letter in your guess will receive one of three symbols:
1. ✓ : The letter is in the word and in the CORRECT position.
2. - : The letter is in the word but in the WRONG position.
3. x : The letter is NOT in the word.

### Example:
Secret Word: BRISK

Guess 1: STORM → Feedback: S(-) T(x) O(x) R(-) M(x)
Guess 2: BRAVE → Feedback: B(✓) R(✓) A(x) V(x) E(x)
Guess 3: BRISK → Feedback: B(✓) R(✓) I(✓) S(✓) K(✓)

### Response Format:
Think through the problem and feedback step by step. Make sure to first add your step by step thought process within <think> </think> tags. Then, return your guessed word in the followi

KeyboardInterrupt: 

In [3]:
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOTrainer, GRPOConfig
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
import reasoning_gym
from reasoning_gym import get_score_answer_fn

# ========== STEP 1: Generate Shortest Path Data ==========

def get_shortest_path_dataset(size=100, seed=42):
    raw = reasoning_gym.create_dataset('family_relationships', size=size, seed=seed)
    records = [
        {"prompt": x["question"], "answer": x["answer"], "metadata": x["metadata"]}
        for x in raw
    ]
    return Dataset.from_list(records), raw  # return both HF dataset and original entries

dataset, raw_entries = get_shortest_path_dataset(size=100, seed=123)

# ========== STEP 2: Load model and tokenizer ==========

model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
base_model = AutoModelForCausalLM.from_pretrained(model_name)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8, lora_alpha=16, lora_dropout=0.05, bias="none",
    target_modules=["q_proj", "v_proj"],
)

model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()

# ========== STEP 3: Reward Function Using RG Verifier ==========

def clean_text(text):
    text = text.lower().strip()
    return re.sub(r'[^a-z\s]', '', text)

def reward_func(prompts, completions, completion_ids, **kwargs):
    metadata = kwargs.get("metadata", [])  # safely get metadata here
    rewards = []

    for i, completion in enumerate(completions):
        idx = completion_ids[i][0] if isinstance(completion_ids[i], list) else completion_ids[i]
        entry = metadata[idx]
        score_fn = get_score_answer_fn(entry["metadata"]["source_dataset"])
        reward = float(score_fn(completion, entry))

        print(f"[Sample {i+1}]")
        print(f"Prompt       : {prompts[i]}")
        print(f"Completion   : {completion.strip()}")
        print(f"Reward Score : {reward}")
        print("="*40)

        rewards.append(reward)

    return rewards



# ========== STEP 4: GRPO Configuration ==========

grpo_config = GRPOConfig(
    output_dir="grpo-shortest-path",
    per_device_train_batch_size=1,
    generation_batch_size=2,
    num_train_epochs=5,
    learning_rate=1e-4,
    num_generations=2,
    logging_steps=5,
    remove_unused_columns=False,
    report_to=[],
    max_completion_length=50,
)

# ========== STEP 5: Initialize Trainer with RG Rewards ==========

trainer = GRPOTrainer(
    model=model,
    args=grpo_config,
    train_dataset=dataset,
    reward_funcs=[lambda prompts, completions, completion_ids, **kwargs: reward_func(prompts, completions, completion_ids, metadata=raw_entries)],
)


# ========== STEP 6: Train and Save ==========

trainer.train()
trainer.save_model("grpo-lora-shortest-path")


INFO 09-10 16:18:22 [__init__.py:241] Automatically detected platform cuda.


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.37it/s]


trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[Sample 1]
Prompt       : Michael is married to Lucy. They have a child called David. David is married to Zoe.

How is Lucy related to Michael? Provide the relationship in one word.
Completion   : A) Mother
B) Child
C) Sibling
D) Aunt
E) None of these

Solution:
--------------------

To understand the relationship between Lucy and Michael, we need to use the information given in
Reward Score : 0.0
[Sample 2]
Prompt       : Michael is married to Lucy. They have a child called David. David is married to Zoe.

How is Lucy related to Michael? Provide the relationship in one word.
Completion   : (2 points)

How is Zoe related to Michael? Provide the relationship in one word. (2 points)

How is David related to Michael? Provide the relationship in one word. (2 points)

<|question_
Reward Score : 0.0


Step,Training Loss
5,0.000000
10,0.000000
15,0.000000
20,0.000000
25,0.000000
30,0.000000
35,0.000000
40,0.000000
45,0.000000
50,0.000000


[Sample 1]
Prompt       : Liam is married to Jennifer. They have a child called Oliver. Oliver is married to Jessica. They have children called Sophie and Luna.

What relation is Jennifer to Liam? Answer with a single word.
Completion   : She is his girlfriend.
Reward Score : 0.0
[Sample 2]
Prompt       : Liam is married to Jennifer. They have a child called Oliver. Oliver is married to Jessica. They have children called Sophie and Luna.

What relation is Jennifer to Liam? Answer with a single word.
Completion   : Answer: wife.
Reward Score : 0.0
[Sample 1]
Prompt       : Charles is married to Karen. They have a child called Alexander. Alexander is married to Sage.

What relation is Sage to Karen? Answer with a single word.
Completion   : Logical Puzzle 1 Answer: Relative
Logical Puzzle 2 Answer: Spouse

Topic: Entertainment
Reward Score : 0.0
[Sample 2]
Prompt       : Charles is married to Karen. They have a child called Alexander. Alexander is married to Sage.

What relation is Sage 

KeyboardInterrupt: 

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from reasoning_gym import create_dataset

def extract_answer(generated_text):
    # Split on 'Answer:'
    parts = generated_text.split("Answer:")
    if len(parts) < 2:
        return ""
    last_part = parts[-1].strip()  # take text after last "Answer:"
    
    # Sometimes the answer ends with new lines or extra text, take first word only
    if len(last_part) == 0:
        return ""
    
    answer_word = last_part.split()[0]
    return answer_word

def safe_extract_answer(text):
    parts = text.split("Answer:")
    if len(parts) < 2:
        return ""
    last_part = parts[-1].strip()
    if not last_part:
        return ""
    words = last_part.split()
    if len(words) == 0:
        return ""
    return words[0]




def main():
    # Parameters
    model_name = "microsoft/phi-2"
    dataset_name = "family_relationships"
    size = 100  # number of samples to test
    seed = 42

    # Load dataset
    raw = create_dataset(dataset_name, size=size, seed=seed)
    samples = raw

    # Load model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    model.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)

    correct = 0

    for i, sample in enumerate(samples):
        question = sample['question']
        answer = sample['answer']

        # Prepare prompt - depends on your task structure
        input_text = question + "\nAnswer:"
        inputs = tokenizer(input_text, return_tensors="pt").to(device)

        # Generate output with model
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=20)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        model_answer = extract_answer(generated_text)

        print(f"Question: {question}")
        print(f"Expected answer: {answer}")
        print(f"Model answer: {model_answer}")
        print("-" * 40)


        # Extract model's answer after "Answer:" prompt
        model_answer = safe_extract_answer(generated_text)

        # Compare with ground truth (simple equality check, can customize)
        if model_answer.lower() == answer.lower():
            correct += 1

        if (i+1) % 10 == 0:
            print(f"Processed {i+1} samples, current accuracy: {correct/(i+1):.2f}")

    print(f"Final accuracy on {size} samples: {correct/size:.2f}")

if __name__ == "__main__":
    main()


/home/athenaik/GRPO_test/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  3.01it/s]
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: John is married to Isabella. They have a child called Edward. Edward is married to Victoria.

What is Isabella to Edward? Respond only with the word that describes their relationship.
Expected answer: mother
Model answer: Mother
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Henry is married to Karen. They have a child called Sebastian. Sebastian is married to Eleanor.

What relation is Henry to Karen? Answer with a single word.
Expected answer: husband
Model answer: 
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Question: Jacob is married to Chloe. They have a child called Samuel. Samuel is married to Margaret. They have a child called Sophia.

What relation is Chloe to Samuel? Answer with a single word.
Expected answer: mother
Model answer: Granddaughter
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: Atlas is married to Nova. They have a child called Phoenix. Phoenix is married to Ava.

How is Atlas related to Phoenix? Provide the relationship in one word.
Expected answer: father
Model answer: Father
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: John is married to Isabella. They have a child called Robert. Robert is married to Aria.

What relation is John to Robert? Answer with a single word.
Expected answer: father
Model answer: Father
----------------------------------------
Question: John is married to Aurora. They have a child called Phoenix. Phoenix is married to Lily. They have children called Henry and Lucy. Theodore is married to Lisa. They have a child called Lily.

How is Phoenix related to Lucy? Provide the relationship in one word.
Expected answer: father
Model answer: Phoenix
----------------------------------------
Processed 100 samples, current accuracy: 0.23
Final accuracy on 100 samples: 0.23
